<a href="https://colab.research.google.com/github/Tiaraindriyani/UDB-Magang/blob/main/apotek_herofarma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas scikit-learn openpyxl

In [2]:
import pandas as pd

df = pd.read_excel('data obat.xlsx')
print(df.head())

  DATA OBAT APOTEK   Unnamed: 1  Unnamed: 2           Unnamed: 3  \
0              NaN          NaN         NaN                  NaN   
1               No    Nama Obat    Kategori             Indikasi   
2                1  Paracetamol   Analgesik  Demam, nyeri ringan   
3                2    Ibuprofen   Analgesik         Neri, radang   
4                3  Amoxicillin  Antibiotik      Infeksi bakteri   

            Unnamed: 4      Unnamed: 5   Unnamed: 6  
0                  NaN             NaN          NaN  
1         Efek Samping  Bentuk Sediaan  Harga (IDR)  
2     Mual, ruam kulit          Tablet     5,000.00  
3  Sakit perut, pusing          Tablet     8,000.00  
4        Diare, alergi          Kapsul    20,000.00  


In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load data
file_path = 'data obat.xlsx'
df = pd.read_excel(file_path, skiprows=2)  # skip metadata rows

# Step 2: Combine relevant text fields
df['gabungan_fitur'] = df['Kategori'].astype(str) + ' ' + df['Indikasi'].astype(str) + ' ' + df['Efek Samping'].astype(str) + ' ' + df['Bentuk Sediaan'].astype(str)

# Step 3: TF-IDF vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['gabungan_fitur'])

# Step 4: Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Step 5: Show similarity matrix
similarity_df = pd.DataFrame(cosine_sim, index=df['Nama Obat'], columns=df['Nama Obat'])
print("\nCosine Similarity Matrix:")
print(similarity_df)

# Step 6: Recommend top-N similar items (example: for Paracetamol)
def get_recommendations(nama_obat, top_n=3):
    idx = df[df['Nama Obat'].str.lower() == nama_obat.lower()].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  # skip itself
    recommended = [(df.iloc[i]['Nama Obat'], score) for i, score in sim_scores]
    return recommended

# Example usage
nama_obat_input = 'Paracetamol'
rekomendasi = get_recommendations(nama_obat_input, top_n=3)
print(f"\nTop rekomendasi mirip dengan {nama_obat_input}:")
for nama, skor in rekomendasi:
    print(f"{nama} (skor: {skor:.2f})")



Cosine Similarity Matrix:
Nama Obat             Paracetamol  Ibuprofen  Amoxicillin  Cetirizine  \
Nama Obat                                                               
Paracetamol              1.000000   0.163143     0.000000    0.022820   
Ibuprofen                0.163143   1.000000     0.000000    0.023965   
Amoxicillin              0.000000   0.000000     1.000000    0.168054   
Cetirizine               0.022820   0.023965     0.168054    1.000000   
Omeprazole               0.000000   0.087410     0.222229    0.000000   
...                           ...        ...          ...         ...   
Sacubitril/Valsartan     0.018821   0.019766     0.000000    0.021909   
Dupilumab                0.000000   0.000000     0.000000    0.000000   
Semaglutide              0.078770   0.000000     0.116063    0.000000   
Osimertinib              0.150377   0.019928     0.107652    0.022090   
Daratumumab              0.000000   0.000000     0.092582    0.000000   

Nama Obat             O